<a href="https://colab.research.google.com/github/jyoti246/alexnet_cuda_cudnn/blob/main/Final_cuda_alex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-r_1z_r2o
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-r_1z_r2o
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=9dbaaab517354efc2000cbe8207d26bc1443be41c4188414760c4841feb54147
  Stored in directory: /tmp/pip-ephem-wheel-cache-3p_15plg/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [ ]:
%%cuda --name alex.cu
#include <cudnn.h>
#include <cublas_v2.h>
#include <cuda_runtime.h>
#include <cstdlib>
#include <cassert>
#include <cstdlib>
#include <iostream>
#include <string>
#include <random>
#include <cmath>
#include <stdio.h>
#include <bits/stdc++.h>
#include <cuda_runtime.h>

using namespace std;

#define BATCH_SIZE 8
#define MAX_THREADS_PER_BLOCK 1024 // according to GTX 1050 Ti

int roundUp(int num, int den)
{

  return((num + den - 1 )/(den));

}

struct convDim_t{

  int Height;
  int Width;
  int Channels;
  int Batch;
};

struct kernelDim_t{

  int kernelSize;
  int kernelHeight;
  int kernelWidth;
  int strideHeight;
  int strideWidth;
  int padHeight;
  int padWidth;
  int dilationHeight;
  int dilationWidth;
};


convDim_t setConvSpecs(int ht, int wd, int ch, int bt){

  convDim_t temp;
  temp.Height = ht;
  temp.Width = wd;
  temp.Channels = ch;
  temp.Batch = bt;

  return temp;
}

kernelDim_t setKernelSpecs(int size, int fheight, int fwidth, int sheight, int swidth, int pheight, int pwidth, int dheight, int dwidth){

  kernelDim_t layerKernel;
  layerKernel.kernelSize = size;
  layerKernel.kernelHeight = fheight;
  layerKernel.kernelWidth = fwidth;
  layerKernel.strideHeight = sheight;
  layerKernel.strideWidth = swidth;
  layerKernel.padHeight = pheight;
  layerKernel.padWidth = pwidth;
  layerKernel.dilationHeight = dheight;
  layerKernel.dilationWidth = dwidth;

  return layerKernel;
}



#define checkCUDNN(expression)                             \
{                                                          \
  cudnnStatus_t status = (expression);                     \
  if (status != CUDNN_STATUS_SUCCESS) {                    \
    std::cerr << "Error on line " << __LINE__ << ": "      \
              << cudnnGetErrorString(status) << std::endl; \
    std::exit(EXIT_FAILURE);                               \
  }                                                        \
}


float alpha = 1.0;
float beta = 0.0;

class ConvLayers{
    
public:
    float *kernelTensor{nullptr};		
    int layerIndex;
	  size_t workspace_bytes{0};

    convDim_t outDims;
    convDim_t inDims;
    kernelDim_t kernelDims;
	    

    cudnnTensorDescriptor_t input_descriptor;
    cudnnTensorDescriptor_t output_descriptor;
    cudnnFilterDescriptor_t kernel_descriptor;
    cudnnConvolutionDescriptor_t convolution_descriptor;
    cudnnConvolutionFwdAlgo_t convolution_algorithm;

    ConvLayers(){}

    ConvLayers(int index, convDim_t inDim, kernelDim_t kdims, convDim_t outDims){

      this->inDims = inDim;
      this->kernelDims = kdims;
      this->layerIndex = index;
	    this->outDims = outDims;
    }

    void buildConvLayer();

    void fwdProp(cudaStream_t stream, cudnnHandle_t cudnn, float *inputTensor, float* &outputTensor, void* &d_workspace);

};

void ConvLayers::buildConvLayer(){
	checkCUDNN(cudnnCreateTensorDescriptor(&input_descriptor));
	checkCUDNN(cudnnSetTensor4dDescriptor(input_descriptor,
										/*format=*/CUDNN_TENSOR_NHWC,
										/*dataType=*/CUDNN_DATA_FLOAT,
										/*batch_size=*/inDims.Batch,
										/*channels=*/inDims.Channels,
										/*image_height=*/inDims.Height,
										/*image_width=*/inDims.Width));



	checkCUDNN(cudnnCreateTensorDescriptor(&output_descriptor));
	checkCUDNN(cudnnSetTensor4dDescriptor(output_descriptor,
										/*format=*/CUDNN_TENSOR_NHWC,
										/*dataType=*/CUDNN_DATA_FLOAT,
										/*batch_size=*/outDims.Batch,
										/*channels=*/outDims.Channels,
										/*image_height=*/outDims.Height,
										/*image_width=*/outDims.Width));   
										
										

	checkCUDNN(cudnnCreateFilterDescriptor(&kernel_descriptor));
	checkCUDNN(cudnnSetFilter4dDescriptor(kernel_descriptor,
										/*dataType=*/CUDNN_DATA_FLOAT,
										/*format=*/CUDNN_TENSOR_NCHW,
										/*out_channels=*/outDims.Channels,
										/*in_channels=*/inDims.Channels,
										/*kernel_height=*/kernelDims.kernelHeight,
										/*kernel_width=*/kernelDims.kernelWidth)); 
										
										

	checkCUDNN(cudnnCreateConvolutionDescriptor(&convolution_descriptor));
	checkCUDNN(cudnnSetConvolution2dDescriptor(convolution_descriptor,
										/*pad_height=*/kernelDims.padHeight,
										/*pad_width=*/kernelDims.padWidth,
										/*vertical_stride=*/kernelDims.strideHeight,
										/*horizontal_stride=*/kernelDims.strideWidth,
										/*dilation_height=*/kernelDims.dilationHeight,
										/*dilation_width=*/kernelDims.dilationWidth,
										/*mode=*/CUDNN_CROSS_CORRELATION,
										/*computeType=*/CUDNN_DATA_FLOAT));


	float h_kernel[outDims.Channels][inDims.Channels][kernelDims.kernelHeight][kernelDims.kernelWidth];
    for (int kernel = 0; kernel < outDims.Channels; ++kernel) {
    for (int channel = 0; channel < inDims.Channels; ++channel) {
        for (int row = 0; row < kernelDims.kernelHeight; ++row) {
        for (int column = 0; column < kernelDims.kernelWidth; ++column) {
            h_kernel[kernel][channel][row][column] = 0.5;
        }
        }
    }
    }

    cudaMalloc(&kernelTensor, sizeof(h_kernel));
    cudaMemcpy(kernelTensor, h_kernel, sizeof(h_kernel), cudaMemcpyHostToDevice); 
}


void ConvLayers::fwdProp(cudaStream_t stream, cudnnHandle_t cudnn, float *inputTensor, float* &outputTensor, void* &d_workspace)
{
	checkCUDNN(cudnnGetConvolutionForwardAlgorithm(cudnn,
                                            input_descriptor,
                                            kernel_descriptor,
                                            convolution_descriptor,
                                            output_descriptor,
                                            CUDNN_CONVOLUTION_FWD_PREFER_FASTEST,
                                            /*memoryLimitInBytes=*/0,
                                            &convolution_algorithm));
                                            

    checkCUDNN(cudnnGetConvolutionForwardWorkspaceSize(cudnn,
                                                    input_descriptor,
                                                    kernel_descriptor,
                                                    convolution_descriptor,
                                                    output_descriptor,
                                                    convolution_algorithm,
                                                    &workspace_bytes));                  
                
    cudaMalloc(&d_workspace, workspace_bytes);       

	int out_bytes = outDims.Batch*outDims.Channels*outDims.Height*outDims.Width*sizeof(float);
    cudaMalloc(&outputTensor, out_bytes);
    cudaMemsetAsync(outputTensor, 0, out_bytes, stream);


    checkCUDNN(cudnnConvolutionForward(cudnn,
                                   &alpha,
                                   input_descriptor,
                                   inputTensor,
                                   kernel_descriptor,
                                   kernelTensor,
                                   convolution_descriptor,
                                   convolution_algorithm,
                                   d_workspace,
                                   workspace_bytes,
                                   &beta,
                                   output_descriptor,
                                   outputTensor));

}


void call_conv1(ConvLayers &convlayer1, cudaStream_t stream, cudnnHandle_t CUDNN, float input[][224][224][3], float output[1][224][224][10])
{
    float *inputTensor;
	float *outputTensor;
	int inp_size = convlayer1.inDims.Height * convlayer1.inDims.Width * convlayer1.inDims.Channels * convlayer1.inDims.Batch * sizeof(float);
    cudaMalloc(&inputTensor, inp_size);
    cudaMemcpyAsync(inputTensor, input, inp_size, cudaMemcpyHostToDevice, stream);

    void *d_workspace;
    convlayer1.fwdProp(stream, CUDNN, inputTensor, outputTensor, d_workspace);

    int out_size =  convlayer1.outDims.Height*convlayer1.outDims.Width*convlayer1.outDims.Channels*convlayer1.outDims.Batch*sizeof(float);
    
    cudaMemcpyAsync(output, outputTensor, out_size, cudaMemcpyDeviceToHost, stream);

    //cout << "CONV1 DONE" << endl;
}

void call_conv2(ConvLayers &convlayer1, cudaStream_t stream, cudnnHandle_t CUDNN, float input[][224][224][10], float output[1][224][224][30])
{
    float *inputTensor;
	float *outputTensor;
	int inp_size = convlayer1.inDims.Height * convlayer1.inDims.Width * convlayer1.inDims.Channels * convlayer1.inDims.Batch * sizeof(float);
    cudaMalloc(&inputTensor, inp_size);
    cudaMemcpyAsync(inputTensor, input, inp_size, cudaMemcpyHostToDevice, stream);

    void *d_workspace;
    convlayer1.fwdProp(stream, CUDNN, inputTensor, outputTensor, d_workspace);

    int out_size =  convlayer1.outDims.Height*convlayer1.outDims.Width*convlayer1.outDims.Channels*convlayer1.outDims.Batch*sizeof(float);
    
    cudaMemcpyAsync(output, outputTensor, out_size, cudaMemcpyDeviceToHost, stream);

    //cout << "CONV2 DONE" << endl;
}


void call_conv3(ConvLayers &convlayer1, cudaStream_t stream, cudnnHandle_t CUDNN, float input[][224][224][30], float output[1][224][224][30])
{
    float *inputTensor;
	float *outputTensor;
	int inp_size = convlayer1.inDims.Height * convlayer1.inDims.Width * convlayer1.inDims.Channels * convlayer1.inDims.Batch * sizeof(float);
    cudaMalloc(&inputTensor, inp_size);
    cudaMemcpyAsync(inputTensor, input, inp_size, cudaMemcpyHostToDevice, stream);

    void *d_workspace;
    convlayer1.fwdProp(stream, CUDNN, inputTensor, outputTensor, d_workspace);

    int out_size =  convlayer1.outDims.Height*convlayer1.outDims.Width*convlayer1.outDims.Channels*convlayer1.outDims.Batch*sizeof(float);
    
    cudaMemcpyAsync(output, outputTensor, out_size, cudaMemcpyDeviceToHost, stream);

    //cout << "CONV3 DONE" << endl;
}

void call_conv4(ConvLayers &convlayer1, cudaStream_t stream, cudnnHandle_t CUDNN, float input[][224][224][30], float output[1][224][224][10])
{
    float *inputTensor;
	float *outputTensor;
	int inp_size = convlayer1.inDims.Height * convlayer1.inDims.Width * convlayer1.inDims.Channels * convlayer1.inDims.Batch * sizeof(float);
    cudaMalloc(&inputTensor, inp_size);
    cudaMemcpyAsync(inputTensor, input, inp_size, cudaMemcpyHostToDevice, stream);

    void *d_workspace;
    convlayer1.fwdProp(stream, CUDNN, inputTensor, outputTensor, d_workspace);

    int out_size =  convlayer1.outDims.Height*convlayer1.outDims.Width*convlayer1.outDims.Channels*convlayer1.outDims.Batch*sizeof(float);
    
    cudaMemcpyAsync(output, outputTensor, out_size, cudaMemcpyDeviceToHost, stream);

    //cout << "CONV4 DONE" << endl;
}

void call_conv5(ConvLayers &convlayer1, cudaStream_t stream, cudnnHandle_t CUDNN, float input[][224][224][10], float output[1][224][224][3])
{
    float *inputTensor;
	float *outputTensor;
	int inp_size = convlayer1.inDims.Height * convlayer1.inDims.Width * convlayer1.inDims.Channels * convlayer1.inDims.Batch * sizeof(float);
    cudaMalloc(&inputTensor, inp_size);
    cudaMemcpyAsync(inputTensor, input, inp_size, cudaMemcpyHostToDevice, stream);

    void *d_workspace;
    convlayer1.fwdProp(stream, CUDNN, inputTensor, outputTensor, d_workspace);

    int out_size =  convlayer1.outDims.Height*convlayer1.outDims.Width*convlayer1.outDims.Channels*convlayer1.outDims.Batch*sizeof(float);
    
    cudaMemcpyAsync(output, outputTensor, out_size, cudaMemcpyDeviceToHost, stream);

    //cout << "CONV5 DONE" << endl;
}


#define num_batches 2
float img[num_batches][BATCH_SIZE][224][224][3];
float output1[num_batches][BATCH_SIZE][224][224][10];
float output2[num_batches][BATCH_SIZE][224][224][30];
float output3[num_batches][BATCH_SIZE][224][224][30];
float output4[num_batches][BATCH_SIZE][224][224][10];
float output5[num_batches][BATCH_SIZE][224][224][3];

void processImg(int index, cudaStream_t stream, cudnnHandle_t cudnn, ConvLayers &convlayer1, ConvLayers &convlayer2, ConvLayers &convlayer3, ConvLayers &convlayer4, ConvLayers &convlayer5)
{
	call_conv1(convlayer1, stream, cudnn, img[index], output1[index]);
	call_conv2(convlayer2, stream, cudnn, output1[index], output2[index]);
	call_conv3(convlayer3, stream, cudnn, output2[index], output3[index]);
	call_conv4(convlayer4, stream, cudnn, output3[index], output4[index]);
	call_conv5(convlayer5, stream, cudnn, output4[index], output5[index]);
}

//channel, height, width
int main(){
	for(int m=0;m<num_batches;++m){
		for(int i=0;i<BATCH_SIZE;++i){
			for(int j=0; j<224;++j){
				for(int k=0;k<224;++k){
					for(int l=0;l<3;++l){
						img[m][i][j][k][l]=0.001;
					}
				}
			}
		}
	}

	convDim_t InputDims1 = setConvSpecs(224, 224, 3, BATCH_SIZE);
	kernelDim_t layerKernel1 = setKernelSpecs(0,3,3,1,1,1,1,1,1);
	convDim_t OutputDims1 = setConvSpecs(224, 224, 10, BATCH_SIZE);
	ConvLayers convlayer1(1, InputDims1, layerKernel1, OutputDims1);
	convlayer1.buildConvLayer();
	
	convDim_t InputDims2 = setConvSpecs(224, 224, 10, BATCH_SIZE);
	kernelDim_t layerKernel2 = setKernelSpecs(0,3,3,1,1,1,1,1,1);
	convDim_t OutputDims2 = setConvSpecs(224, 224, 30, BATCH_SIZE);
	ConvLayers convlayer2(2, InputDims2, layerKernel2, OutputDims2);
	convlayer2.buildConvLayer();

	convDim_t InputDims3 = setConvSpecs(224, 224, 30, BATCH_SIZE);
	kernelDim_t layerKernel3 = setKernelSpecs(0,3,3,1,1,1,1,1,1);
	convDim_t OutputDims3 = setConvSpecs(224, 224, 30, BATCH_SIZE);
	ConvLayers convlayer3(3, InputDims3, layerKernel3, OutputDims3);
	convlayer3.buildConvLayer();

	convDim_t InputDims4 = setConvSpecs(224, 224, 30, BATCH_SIZE);
	kernelDim_t layerKernel4 = setKernelSpecs(0,3,3,1,1,1,1,1,1);
	convDim_t OutputDims4 = setConvSpecs(224, 224, 10, BATCH_SIZE);
	ConvLayers convlayer4(4, InputDims4, layerKernel4, OutputDims4);
	convlayer4.buildConvLayer();

	convDim_t InputDims5 = setConvSpecs(224, 224, 10, BATCH_SIZE);
	kernelDim_t layerKernel5 = setKernelSpecs(0,3,3,1,1,1,1,1,1);
	convDim_t OutputDims5 = setConvSpecs(224, 224, 3, BATCH_SIZE);
	ConvLayers convlayer5(5, InputDims5, layerKernel5, OutputDims5);
	convlayer5.buildConvLayer();
  
  cudaDeviceSynchronize();

  for(int nstreams=1; nstreams<=8;++nstreams)
  {
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    cudaStream_t stream[nstreams];
    cudnnHandle_t cudnn[nstreams];
    for(int i=0;i<nstreams;++i){
      cudaStreamCreate(&stream[i]);
      checkCUDNN(cudnnCreate(&cudnn[i]));
      cudnnSetStream(cudnn[i], stream[i]);
    }

    cudaEventRecord(start);
    for(int i=0;i<num_batches;++i)
    {
      cudaStreamSynchronize(stream[i%nstreams]);
      processImg(i, stream[i%nstreams], cudnn[i%nstreams], convlayer1, convlayer2, convlayer3, convlayer4, convlayer5);
      //cout<<"Image "<<i<<" dispatched."<<endl;
    }
    cudaDeviceSynchronize();
   
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float elapsedtime;
    cudaEventElapsedTime(&elapsedtime, start, stop);
    cout << "Nstreams "<<nstreams<<"ELAPSED TIME: " << elapsedtime << endl;
  }
	cudaDeviceSynchronize();
	return 0;
}

'File written in /content/src/alex.cu'

In [ ]:
!nvcc /content/src/alex.cu `pkg-config --cflags --libs opencv` -lcudnn -lcublas -lopencv_imgcodecs -lopencv_imgproc -lopencv_core -pg -std=c++11 -o /content/src/alex

In [ ]:
!/content/src/alex

Nstreams 1ELAPSED TIME: 758.375
Nstreams 2ELAPSED TIME: 435.611
Nstreams 3ELAPSED TIME: 373.12
Nstreams 4ELAPSED TIME: 349.3
Nstreams 5ELAPSED TIME: 356.406
Nstreams 6ELAPSED TIME: 355.278
Nstreams 7ELAPSED TIME: 355.314
Nstreams 8ELAPSED TIME: 352.136
